v0.91

> TODO:
- remove LOAD_SAVED_DATA
- write in the beginning gthat this notepbook runs for !3h
- save only last checkpoint csv, rename the final csv
- wl_bul_wikipedia_2021_1M-sentences_part1.csv to be one part, use git-lfs
- remove the warning
- add descriptions + fix headings
- remove yellow
- extract all functions to py files
- run locally
- check typos

In [ ]:
!pip install ufal.udpipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.8/936.8 kB 16.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import os
import pandas as pd
import sys
import urllib.request
import warnings

import re
import time

from copy import copy, deepcopy
from ufal.udpipe import Model, Pipeline, ProcessingError

In [ ]:
!if [ ! -f "/content/helpers/extract_features.py" ]; then wget -P helpers/ https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/extract_features.py; fi
!if [ ! -f "/content/helpers/extract_xpostag.py" ]; then wget -P helpers/ https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/extract_xpostag.py; fi


--2025-01-21 15:42:16--  https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/extract_features.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1262 (1.2K) [text/plain]
Saving to: ‘helpers/extract_features.py’

extract_features.py 100%[===================>]   1.23K  --.-KB/s    in 0s      

2025-01-21 15:42:17 (60.9 MB/s) - ‘helpers/extract_features.py’ saved [1262/1262]

--2025-01-21 15:42:17--  https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/extract_xpostag.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.19

In [ ]:
IS_GUEST = False
LOAD_SAVED_DATA = True # If False, running this notebook takes ~ 3 hours (Google Colab/CPU).
LOAD_SAVED_DATA_TMP = True

In [ ]:
if IS_GUEST:
    root_dir = '.'
else:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    root_dir = "/content/drive/MyDrive/softuni/the-grammar-whisperer"

data_raw_dir = f"{root_dir}/data/raw"
data_clean_dir = f"{root_dir}/data/clean"
data_processed_dir = f"{root_dir}/data/processed"
helpers_dir = f"{root_dir}/helpers"

if root_dir not in sys.path:
    sys.path.append(root_dir)
    sys.path.append(helpers_dir)

wl_wiki_2021_1M_part1 = f'{data_clean_dir}/wl_bul_wikipedia_2021_1M-sentences_part1.csv'
wl_wiki_2021_1M_part2 = f'{data_clean_dir}/wl_bul_wikipedia_2021_1M-sentences_part2.csv'

Mounted at /content/drive


In [ ]:
from extract_features import extract_features
from extract_xpostag import extract_xpostag

In [ ]:
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning) # 🧡

In [ ]:
pd.set_option('display.max_colwidth', 500)

###### Functions for saving and loading a checkpoint (❤️move to .py file)


In [ ]:
import os
import glob
import re

def save_checkpoint(base_filename, data, N=5):
    """
    Save a checkpoint file while maintaining only the last N versions.

    Args:
        base_filename (str): Base name of the file (e.g., 'model.pt')
        data: The data to save
    """
    # Get existing checkpoints
    name, ext = os.path.splitext(base_filename)
    pattern = f"{name}_checkpoint[0-9]*{ext}"
    existing_files = glob.glob(pattern)

    # Extract checkpoint numbers
    numbers = []
    for file in existing_files:
        match = re.search(rf"{name}_checkpoint(\d+){ext}", file)
        if match:
            numbers.append(int(match.group(1)))

    # Determine next checkpoint number
    next_num = 1 if not numbers else max(numbers) + 1

    # Save new checkpoint
    checkpoint_name = f"{name}_checkpoint{next_num}{ext}"
    data.to_csv(checkpoint_name, index=False)

    # Remove old checkpoints if more than N
    if len(existing_files) >= N:
        checkpoint_files = [(f, int(re.search(rf"{name}_checkpoint(\d+){ext}", f).group(1)))
                          for f in existing_files]
        checkpoint_files.sort(key=lambda x: x[1])

        # Remove oldest files until only 4 remain (plus the new one we just added)
        while len(checkpoint_files) >= N:
            os.remove(checkpoint_files[0][0])
            checkpoint_files.pop(0)

    return checkpoint_name

def load_checkpoint(base_filename, sep=','):
    """
    Load the latest checkpoint file.

    Args:
        base_filename (str): Base name of the file (e.g., 'model.pt')

    Returns:
        The loaded data from the latest checkpoint
    """
    name, ext = os.path.splitext(base_filename)
    pattern = f"{name}_checkpoint[0-9]*{ext}"
    existing_files = glob.glob(pattern)

    if not existing_files:
        if os.path.exists(base_filename):
            return pd.read_csv(base_filename, sep=sep), base_filename
        else:
            return None, None

    # Find the highest checkpoint number
    checkpoint_files = [(f, int(re.search(rf"{name}_checkpoint(\d+){ext}", f).group(1)))
                       for f in existing_files]
    latest_file = max(checkpoint_files, key=lambda x: x[1])[0]

    # Your existing load function here, e.g.:
    return pd.read_csv(latest_file, sep=sep), latest_file

## Add lemmas and pos tagging to sentences

In [ ]:
start_time = time.time()
print(f"Execution time: {time.time() - start_time} seconds")

Execution time: 7.009506225585938e-05 seconds


###### Load clean csv with sentences

In [ ]:
df1, df2 = pd.read_csv(wl_wiki_2021_1M_part1, sep='|'), pd.read_csv(wl_wiki_2021_1M_part2, sep='|')

In [ ]:
df_clean = pd.concat([df1, df2], ignore_index=True)  # Combine and regenerate a new index
del df1 # delete so we don't accidentally refer to it in subsequent code
del df2

In [ ]:
for x in df_clean.iloc[0:10]['sentence']:
    print(x)

Става световен шампион в тежка категория през 2007 г.
Лечението е трудно и продължително.
Въпросните тестови случаи се сформират на база спецификациите и изискванията, т.е. въз основа на това за какво е предназначена дадено приложение.
През този сезон 2009 клубът носи името "Гаосюн Яоти".
От всички дъгови бои при колумбийските географското разпространение е най-северно.
Официално днес носи името "Свети Модест" .
Достъпът до двигателя се осъществява през люк в десния борд.
Между българите има повече от 1 млн. мохамедани.
Млекопроизводството и отглеждането на едър рогат добитък и свине също става важно да селскостопанската икономика на района.
Възможни са проблеми, когато изследването на психичния статус се прилага, когато клиницистът и пациентът са от различен културен произход.


In [ ]:
df_clean.shape

(787244, 1)

##### ➡️Split to words and lemmas using UDPipe

In [ ]:
# Download the UDPipe Bulgarian model
model_path = "bulgarian-btb-ud-2.5-191206.udpipe"

if not os.path.exists(model_path):
    model_url_udpipe = "https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/bulgarian-btb-ud-2.5-191206.udpipe?sequence=6&isAllowed=y"
    urllib.request.urlretrieve(model_url_udpipe, model_path)

model = Model.load(model_path)
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

###### Function to process one sentence into pos, features, etc.

In [ ]:
def udpipe_to_spacy(text):
    """Parses a CoNLL-U formatted string into a spaCy Doc object."""
    doc_str = pipeline.process(text)

    lines = doc_str.strip().splitlines()
    words = []
    lemmas = []
    spaces = []
    pos_tags = []
    morph_tags = []
    features = []
    dep_rels = []

    for line in lines:
        if line.startswith("#") or not line.strip():
            continue

        parts = line.split("\t")
        index, word, lemma, pos, tag, feats, head, dep_rel, _, misc = parts

        words.append(word)
        lemmas.append(lemma)
        pos_tags.append(pos)  # POS tag (simpler POS category)
        morph_tags.append(tag)  # Detailed morphological tag
        features.append(feats)
        dep_rels.append(dep_rel)

        if "SpaceAfter=No" in misc:
            spaces.append(False)
        else:
            spaces.append(True)

    return words, lemmas, pos_tags, morph_tags, features, dep_rels, len(words)

###### Process the entire clean dataframe in chunks using checkpoints

In [ ]:
def find_unprocessed_indices(df, col):
    """Finds the indices of rows in the DataFrame where the column is NaN, assuming those rows are unprocessed."""

    # Check if the column exists
    if col not in df.columns:
        return df.index.tolist()

    # Find unprocessed rows where the value in the column is NaN
    unprocessed_mask = df[col].isna()
    unprocessed_indices = df[unprocessed_mask].index.tolist()

    return unprocessed_indices

def process_dataframe_with_checkpoints(_df, process_func, chunk_size=1000, output_path='final_output.csv'):
    """
    Process a DataFrame in chunks with checkpoint capability using vectorized operations.

    Args:
        _df: Input DataFrame
        process_func: Function to apply to each chunk
        chunk_size: Number of rows to process in each chunk
        checkpoint_path: Path to save checkpoint files
        output_path: Path to save final output
    """

    df, checkpoint_filename = load_checkpoint(output_path)
    if df is None:
        df = _df.copy()
        print("Starting fresh processing...")
    else:
        print(f"Loaded checkpoint {checkpoint_filename} with {len(df)} rows")

    if False:
        if os.path.exists(checkpoint_path):
            df = pd.read_csv(checkpoint_path)
            print(f"Loaded checkpoint with {len(df)} rows")
        else:
            df = _df.copy()
            print("Starting fresh processing")

    # Find which rows are not processed yet
    # Assuming if 'words' column is NaN, the row hasn't been processed
    # unprocessed_mask = df['words'].isna()
    # unprocessed_indices = df[unprocessed_mask].index
    unprocessed_indices = find_unprocessed_indices(df, 'words')

    if len(unprocessed_indices) == 0:
        print("All rows already processed")
        df.to_csv(output_path, index=False)
        return df

    total_chunks = (df.shape[0] + chunk_size - 1) // chunk_size

    current_chunks = (len(unprocessed_indices) + chunk_size - 1) // chunk_size

    for current_chunk_num in range(current_chunks):
        try:
            chunk_start = current_chunk_num * chunk_size
            chunk_end = min((current_chunk_num + 1) * chunk_size, len(unprocessed_indices))
            current_indices = unprocessed_indices[chunk_start:chunk_end]
            start_idx = current_indices[0]
            end_idx = current_indices[-1]

            print(f"Processing chunk {current_chunk_num + 1}/{current_chunks} (indices [{start_idx}-{end_idx}]/{df.shape[0]})")

            # Process current chunk in a vectorized way
            chunk_sentences = df.loc[current_indices, 'sentence']
            results = chunk_sentences.apply(process_func).apply(pd.Series)

            # Update the dataframe with processed results
            results.columns = ["words", "lemmas", "pos", "morph", "features", "dep", "n_words"]
            df.loc[current_indices, results.columns] = results

            # Save checkpoint after each chunk
            # df.to_csv(checkpoint_path, index=False)
            latest_filename = save_checkpoint(output_path, df)
            print(f"Saved checkpoint {latest_filename}")

        except KeyboardInterrupt:
            print("\nProcessing interrupted. Progress saved in checkpoint file.")
            # df.to_csv(checkpoint_path, index=False)
            save_checkpoint(output_path, df)
            return df
        except Exception as e:
            print(f"Error processing chunk: {e}")
            # df.to_csv(checkpoint_path, index=False)
            save_checkpoint(output_path, df)
            raise

    # All processing complete, save final output
    unprocessed_indices = find_unprocessed_indices(df, 'words')
    if len(unprocessed_indices) == 0:
        print("Processing complete")
        df.to_csv(output_path, index=False)

    return df

In [ ]:
chunk_size = 10000
output_path = f'{data_processed_dir}/sent_wikipedia_nlp_features_v1.csv'

In [ ]:
# this runs for ~ 3 hours on Google Colab/CPU
if not LOAD_SAVED_DATA:
    df_processed = process_dataframe_with_checkpoints(
        df_clean,
        udpipe_to_spacy,
        chunk_size=chunk_size,
        output_path=output_path
    )
else:
    df_processed = pd.read_csv(output_path)

In [ ]:
df_clean.shape, df_processed.shape, df_processed.columns

((787244, 1),
 (787244, 9),
 Index(['sentence', 'words', 'lemmas', 'pos', 'morph', 'features', 'dep',
        'n_words', 'n_lemmas'],
       dtype='object'))

In [ ]:
df_processed.head(3)

,sentence,words,lemmas,pos,morph,features,dep,n_words,n_lemmas
0,Става световен шампион в тежка категория през 2007 г.,"['Става', 'световен', 'шампион', 'в', 'тежка', 'категория', 'през', '2007', 'г.']","['ставам', 'световен', 'шампион', 'в', 'тежък', 'категория', 'през', '2007', 'година']","['VERB', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN']","['Vniif-r3s', 'Amsi', 'Ncmsi', 'R', 'Afsi', 'Ncfsi', 'R', 'Mofsi', 'Ncfsi']","['Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'Definite=Ind|Degree=Pos|Gender=Masc|Number=Sing', 'Definite=Ind|Gender=Masc|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Fem|Number=Sing', 'Definite=Ind|Gender=Fem|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Fem|Number=Sing|NumType=Ord', 'Definite=Ind|Gender=Fem|Number=Sing']","['root', 'amod', 'obj', 'case', 'amod', 'obl', 'case', 'amod', 'obl']",9.0,9.0
1,Лечението е трудно и продължително.,"['Лечението', 'е', 'трудно', 'и', 'продължително', '.']","['Лечение', 'съм', 'трудно', 'и', 'продължително', '.']","['NOUN', 'AUX', 'ADJ', 'CCONJ', 'ADJ', 'PUNCT']","['Ncnsd', 'Vxitf-r3s', 'Ansi', 'Cp', 'Ansi', 'punct']","['Definite=Def|Gender=Neut|Number=Sing', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing', '_']","['nsubj', 'cop', 'root', 'cc', 'conj', 'punct']",6.0,6.0
2,"Въпросните тестови случаи се сформират на база спецификациите и изискванията, т.е. въз основа на това за какво е предназначена дадено приложение.","['Въпросните', 'тестови', 'случаи', 'се', 'сформират', 'на', 'база', 'спецификациите', 'и', 'изискванията', ',', 'т.е.', 'въз', 'основа', 'на', 'това', 'за', 'какво', 'е', 'предназначена', 'дадено', 'приложение', '.']","['Въпросен', 'тестов', 'случай', 'се', 'сформирам', 'на', 'база', 'спецификация', 'и', 'изискване', ',', 'т.е.', 'въз', 'основа', 'на', 'този', 'за', 'какъв', 'съм', 'предназнача', 'дам-(се)', 'приложение', '.']","['ADJ', 'ADJ', 'NOUN', 'PRON', 'VERB', 'ADP', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'CCONJ', 'ADP', 'NOUN', 'ADP', 'PRON', 'ADP', 'DET', 'AUX', 'VERB', 'ADJ', 'NOUN', 'PUNCT']","['A-pd', 'A-pi', 'Ncmpi', 'Ppxta', 'Vpitf-r3p', 'R', 'Ncfsi', 'Ncfpd', 'Cp', 'Ncnpd', 'punct', 'Cc', 'R', 'Ncfsi', 'R', 'Pde-os-n', 'R', 'Pia--s-n', 'Vxitf-r3s', 'Vpptcv--sfi', 'Ansi', 'Ncnsi', 'punct']","['Definite=Def|Degree=Pos|Number=Plur', 'Definite=Ind|Degree=Pos|Number=Plur', 'Definite=Ind|Gender=Masc|Number=Plur', 'Case=Acc|PronType=Prs|Reflex=Yes', 'Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', '_', 'Definite=Ind|Gender=Fem|Number=Sing', 'Definite=Def|Gender=Fem|Number=Plur', '_', 'Definite=Def|Gender=Neut|Number=Plur', '_', '_', '_', 'Definite=Ind|Gender=Fem|Number=Sing', '_', 'Case=Nom|Gender=Neut|Number=Sing|PronType=Dem', '_', 'Gender=Neut|Number=Si...","['amod', 'amod', 'nsubj', 'expl', 'ccomp', 'case', 'obl', 'nsubj:pass', 'cc', 'conj', 'punct', 'cc', 'case', 'fixed', 'fixed', 'obl', 'case', 'obl', 'aux:pass', 'root', 'amod', 'nsubj:pass', 'punct']",23.0,23.0


###### Add gender, number, person

In [ ]:
results = df_processed["features"].apply(extract_features).apply(pd.Series)

results.columns = ["gender", "number", "person"]
df_processed[results.columns] = results

In [ ]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787244 entries, 0 to 787243
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   sentence  787244 non-null  object 
 1   words     787244 non-null  object 
 2   lemmas    787244 non-null  object 
 3   pos       787244 non-null  object 
 4   morph     787244 non-null  object 
 5   features  787244 non-null  object 
 6   dep       787244 non-null  object 
 7   n_words   787244 non-null  float64
 8   n_lemmas  787244 non-null  float64
 9   gender    787244 non-null  object 
 10  number    787244 non-null  object 
 11  person    787244 non-null  object 
dtypes: float64(2), object(10)
memory usage: 72.1+ MB


In [ ]:
df_processed.head(3)

,sentence,words,lemmas,pos,morph,features,dep,n_words,n_lemmas,gender,number,person
0,Става световен шампион в тежка категория през 2007 г.,"['Става', 'световен', 'шампион', 'в', 'тежка', 'категория', 'през', '2007', 'г.']","['ставам', 'световен', 'шампион', 'в', 'тежък', 'категория', 'през', '2007', 'година']","['VERB', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN']","['Vniif-r3s', 'Amsi', 'Ncmsi', 'R', 'Afsi', 'Ncfsi', 'R', 'Mofsi', 'Ncfsi']","['Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'Definite=Ind|Degree=Pos|Gender=Masc|Number=Sing', 'Definite=Ind|Gender=Masc|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Fem|Number=Sing', 'Definite=Ind|Gender=Fem|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Fem|Number=Sing|NumType=Ord', 'Definite=Ind|Gender=Fem|Number=Sing']","['root', 'amod', 'obj', 'case', 'amod', 'obl', 'case', 'amod', 'obl']",9.0,9.0,.MM.FF.FF,SSS.SS.SS,3........
1,Лечението е трудно и продължително.,"['Лечението', 'е', 'трудно', 'и', 'продължително', '.']","['Лечение', 'съм', 'трудно', 'и', 'продължително', '.']","['NOUN', 'AUX', 'ADJ', 'CCONJ', 'ADJ', 'PUNCT']","['Ncnsd', 'Vxitf-r3s', 'Ansi', 'Cp', 'Ansi', 'punct']","['Definite=Def|Gender=Neut|Number=Sing', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing', '_', 'Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing', '_']","['nsubj', 'cop', 'root', 'cc', 'conj', 'punct']",6.0,6.0,N.N.N.,SSS.S.,.3....
2,"Въпросните тестови случаи се сформират на база спецификациите и изискванията, т.е. въз основа на това за какво е предназначена дадено приложение.","['Въпросните', 'тестови', 'случаи', 'се', 'сформират', 'на', 'база', 'спецификациите', 'и', 'изискванията', ',', 'т.е.', 'въз', 'основа', 'на', 'това', 'за', 'какво', 'е', 'предназначена', 'дадено', 'приложение', '.']","['Въпросен', 'тестов', 'случай', 'се', 'сформирам', 'на', 'база', 'спецификация', 'и', 'изискване', ',', 'т.е.', 'въз', 'основа', 'на', 'този', 'за', 'какъв', 'съм', 'предназнача', 'дам-(се)', 'приложение', '.']","['ADJ', 'ADJ', 'NOUN', 'PRON', 'VERB', 'ADP', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'CCONJ', 'ADP', 'NOUN', 'ADP', 'PRON', 'ADP', 'DET', 'AUX', 'VERB', 'ADJ', 'NOUN', 'PUNCT']","['A-pd', 'A-pi', 'Ncmpi', 'Ppxta', 'Vpitf-r3p', 'R', 'Ncfsi', 'Ncfpd', 'Cp', 'Ncnpd', 'punct', 'Cc', 'R', 'Ncfsi', 'R', 'Pde-os-n', 'R', 'Pia--s-n', 'Vxitf-r3s', 'Vpptcv--sfi', 'Ansi', 'Ncnsi', 'punct']","['Definite=Def|Degree=Pos|Number=Plur', 'Definite=Ind|Degree=Pos|Number=Plur', 'Definite=Ind|Gender=Masc|Number=Plur', 'Case=Acc|PronType=Prs|Reflex=Yes', 'Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', '_', 'Definite=Ind|Gender=Fem|Number=Sing', 'Definite=Def|Gender=Fem|Number=Plur', '_', 'Definite=Def|Gender=Neut|Number=Plur', '_', '_', '_', 'Definite=Ind|Gender=Fem|Number=Sing', '_', 'Case=Nom|Gender=Neut|Number=Sing|PronType=Dem', '_', 'Gender=Neut|Number=Si...","['amod', 'amod', 'nsubj', 'expl', 'ccomp', 'case', 'obl', 'nsubj:pass', 'cc', 'conj', 'punct', 'cc', 'case', 'fixed', 'fixed', 'obl', 'case', 'obl', 'aux:pass', 'root', 'amod', 'nsubj:pass', 'punct']",23.0,23.0,..M...FF.N...F.N.N.FNN.,PPP.P.SP.P...S.S.SSSSS.,....3.............3....


###### Add case

In [ ]:
results = df_processed["morph"].apply(extract_xpostag).apply(pd.Series)

results.columns = ["case"]
df_processed[results.columns] = results

In [ ]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787244 entries, 0 to 787243
Data columns (total 13 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   sentence  787244 non-null  object 
 1   words     787244 non-null  object 
 2   lemmas    787244 non-null  object 
 3   pos       787244 non-null  object 
 4   morph     787244 non-null  object 
 5   features  787244 non-null  object 
 6   dep       787244 non-null  object 
 7   n_words   787244 non-null  float64
 8   n_lemmas  787244 non-null  float64
 9   gender    787244 non-null  object 
 10  number    787244 non-null  object 
 11  person    787244 non-null  object 
 12  case      787244 non-null  object 
dtypes: float64(2), object(11)
memory usage: 78.1+ MB


In [ ]:
output_path = f'{data_processed_dir}/sent_wikipedia_nlp_features_v2.csv'
df_processed.to_csv(output_path, index=False)

###### Convert to lowercase

In [ ]:
columns_to_lower = ['sentence', 'words', 'lemmas']
df_processed[columns_to_lower] = df_processed[columns_to_lower].apply(lambda x: x.str.lower())

###### Remove columns with repeated information

In [ ]:
df_processed_no_repeat = df_processed.drop(columns=['morph', 'features', 'dep', 'n_lemmas'])

output_path = f'{data_processed_dir}/sent_wikipedia_nlp_features_final.csv'
df_processed_no_repeat.to_csv(output_path, index=False)

In [ ]:
!ls -alh {data_processed_dir}

total 6.5G
-rw------- 1 root root  96M Jan 21 14:05 bg_vocabulary_v1.csv
-rw------- 1 root root  76M Jan 21 14:24 bg_vocabulary_v2.csv
-rw------- 1 root root 663M Jan 21 15:55 sent_wikipedia_nlp_features_final.csv
-rw------- 1 root root 100M Jan 20 09:50 sent_wikipedia_nlp_features.part01.rar
-rw------- 1 root root  56M Jan 20 09:50 sent_wikipedia_nlp_features.part02.rar
-rw------- 1 root root 1.3G Jan 21 11:11 sent_wikipedia_nlp_features_v1.csv
-rw------- 1 root root 1.4G Jan 21 15:48 sent_wikipedia_nlp_features_v2.csv
-rw------- 1 root root 868K Jan 21 12:38 sent_wikipedia_nlp_features_v3_1000_tmp.csv
-rw------- 1 root root 663M Jan 21 11:58 sent_wikipedia_nlp_features_v3.csv
-rw------- 1 root root 1.5G Jan 20 20:40 sent_wikipedia_nlp_more_features.csv
-rw------- 1 root root 850M Jan 20 20:44 sent_wikipedia_nlp_more_features_no_repeat.csv
